In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/whats-cooking-kernels-only/sample_submission.csv.zip
/kaggle/input/whats-cooking-kernels-only/train.json
/kaggle/input/whats-cooking-kernels-only/test.json


In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

In [3]:
train = json.load(open('/kaggle/input/whats-cooking-kernels-only/train.json'))
test = json.load(open('/kaggle/input/whats-cooking-kernels-only/test.json'))
train_doc = [" ".join(doc['ingredients']).lower() for doc in train]
test_doc = [" ".join(doc['ingredients']).lower() for doc in test]

# Label Encoding of y - the target kinds of cuisine
y_train = [doc['cuisine'] for doc in train]
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_train

array([ 6, 16,  4, ...,  8,  3, 13])

In [4]:
# TFIDF statiscic applying to the data - resulting in sparse matrix
tfidf = TfidfVectorizer(binary=True)
X_train = tfidf.fit_transform(train_doc)
X_test = tfidf.transform(test_doc)
X_train

<39774x3010 sparse matrix of type '<class 'numpy.float64'>'
	with 761951 stored elements in Compressed Sparse Row format>

In [5]:
#Support Vector Machine
svc = SVC(C=100, gamma=0.9, coef0=1, tol=0.001, decision_function_shape='ovr')
model_svc = OneVsRestClassifier(svc, n_jobs=1)
model_svc.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(C=100, coef0=1, gamma=0.9), n_jobs=1)

In [6]:
# Predictions based on supported vector machine
y_test = model_svc.predict(X_test)
print(y_test)
y_pred = lb.inverse_transform(y_test)
print(y_pred)

[ 8 16  9 ...  9 16 13]
['irish' 'southern_us' 'italian' ... 'italian' 'southern_us' 'mexican']


In [7]:
# Saving results to file
test_id = [doc['id'] for doc in test]
sub = pd.DataFrame({'id': test_id, 'cuisine': y_pred}, columns=['id', 'cuisine'])
sub.to_csv('submission.csv', index=False)